<a href="https://colab.research.google.com/github/happyhoo97/Skin-Cancer-Dectection-Application/blob/main/Cancer_Dectection_MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>